# 4-D STEM Analysis Using Pyxem 


## Data Inspection- Preprocessing - Unsupervised ML - Lazy Processing - Orientation Analysis 


### Carter Francis | University of Wisconsin Madison | May 24th 2023

Pyxem Introduction:
-------------------

`Pyxem` was first started in 2016 by Duncan Johnston.  Since then it has been continually developed alongside hyperspy. Below is a very simplified dependancy tree for `pyxem`. We inherit quite a bit of functionality from upstream packages and ascribe strongly to the ideal that if we can upstream code to make it available to a wider audience we should!

We are always looking for more people to join our team [here](https://github.com/pyxem/pyxem)!  

<img style="left" src="imgs/DependancyTree.svg">

### Our Focus/ Goals:

1. Provide Scalable analysis for pixelated(mostly 4-D STEM) diffraction 
    - Pyxem (and hyperspy) scales from single core --> Multi-core single machiene --> Multicore Distributed Computing!
    - Fast I-O allows streaming and processing of TB sized datasets in under a ___minute___!
    - Hyperspy is ___Fast___! I mean like really fast. If you don't believe me try running using the dask distributed backend. 
2. Provide End to End workflows without limiting functionality.
    - A focus on documentation and example notebooks keeping the internal `pyxem` code simple and easy to maintain and grow.
3. Testing Testing Testing!
    - Pyxem is focused on test driven development which limits the number of bugs and helps us to understand why bugs arise when they do
    - While not perfect this helps us to know that updates won't cause functionality to fail.
4. Learning and Teaching!
    - Drop by and say hi on github.  Make an [issue](https://github.com/pyxem/pyxem/issues) for a feature you would like, add some code you find helpful. 
    - Even if you are just trying something out or need help we are always happy to help!

Data Introduction
-----------------
This data is a set of MgO nanocrystals on a lacy Carbon sample.  It is a pretty (very small) 4-D STEM dataset that I could load into RAM. But I personally still like to run everything lazily (and using the distributed backend) for a couple of reasons:

1. I'm Lazy (why shouldn't my data be):
    - I like things to load imediately and don't like waiting around
    - Lazy makes things like running in parallel EASY
2. Lazy means better parallelization (and it's Fast!):
    - Lazy data is already set up to run in parallel so you get better control
    - I love the dask-dashboard (and you should too)
3. One workflow, Any Size of Data
    - You can very easily take the same code. Move it to a cluster or a HPC cluster :)
    - Lazy workflows mean faster iteration, faster discovery which means more experiments. 


## Contents

1. <a href='#loa'> Loading & Inspection</a>
2. <a href='#cal'> Alignment & Calibration</a>
3. <a href='#vdf'> Virtual Diffraction Imaging</a>
4. <a href='#ml'> Machine Learning SPED Data</a>
5. <a href='#vec'> Peak Finding and Segmentation</a>

## 0. Import pyxem and other required libraries

In [1]:
# I like to print out the current version for hyperspy that I am using just in case I come back to a
# notbook a many years later and things change slightly
import hyperspy
print(hyperspy.__version__)
import pyxem
print(pyxem.__version__)

1.7.5


0.15.1


In [2]:
# Changing the matplotlib background will give you interactive 
%matplotlib qt5
#%matplotlib inline
#%matplotlib widget # for plotting when running remotely on a cluster etc.
import hyperspy.api as hs
import pyxem as pxm
import numpy as np

In [3]:
# Starting up a distributed Cluster locally 
# You don't have to do this but it helps to visualize what is happening
from dask.distributed import Client
client = Client()  # set up local cluster on your laptop
client

INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:39491
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:34341'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:42453'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:43749'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:39001'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:35847', name: 1, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:35847
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:50116
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:44797', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:44797
INFO:distributed.core:Starting est

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 15.43 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39491,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 15.43 GiB
Comm: tcp://127.0.0.1:44797,Total threads: 3
Dashboard: http://127.0.0.1:42085/status,Memory: 3.86 GiB
Nanny: tcp://127.0.0.1:34341,


<a id='loa'></a>

##  1. Loading and Inspection

Load the SPED data acquired from the nano-crystals using hyperspy.

`Note: Because pyxem extends hyperspy this happens automatically!`

In [4]:
import hyperspy.api as hs
dp = hs.load("data/mgo_nanoparticles.zspy", lazy=True)

In [5]:
# lets just look at the data 
# display(dp) also works

<LazyElectronDiffraction2D, title: , dimensions: (109, 114|144, 144)>

In [6]:
# Lets change the title here so it shows up when we load the dataset

In [7]:
# Then we can display the data strucuture again. 

<LazyElectronDiffraction2D, title: MgO Nano-Crystals, dimensions: (109, 114|144, 144)>

Inspect the dp object

In [9]:
# currently the navigator isn't set so in order to plot this we have to "create" one by
# summing the entire dataset. Look at the distributed task-stream to see all of the chunks are
# loaded and then a Summed navigator is created.  This isn't very efficient (or lazy!)


# if you rerun this cell the navigator is saved (Yay) and it takes much less time to plot the data!

<ElectronDiffraction2D, title: MgO Nano-Crystals, dimensions: (|109, 114)>

├── General
│   ├── FileIO
│   │   ├── 0
│   │   │   ├── hyperspy_version = 1.7.5
│   │   │   ├── io_plugin = hyperspy.io_plugins.hspy
│   │   │   ├── operation = load
│   │   │   └── timestamp = 2023-05-19T12:48:34.428982-05:00
│   │   ├── 1
│   │   │   ├── hyperspy_version = 1.7.5
│   │   │   ├── io_plugin = hyperspy.io_plugins.zspy
│   │   │   ├── operation = save
│   │   │   └── timestamp = 2023-05-19T12:48:37.501801-05:00
│   │   ├── 2
│   │   │   ├── hyperspy_version = 1.7.5
│   │   │   ├── io_plugin = hyperspy.io_plugins.zspy
│   │   │   ├── operation = save
│   │   │   └── timestamp = 2023-05-19T12:49:11.044073-05:00
│   │   ├── 3
│   │   │   ├── hyperspy_version = 1.7.5
│   │   │   ├── io_plugin = hyperspy.io_plugins.zspy
│   │   │   ├── operation = save
│   │   │   └── timestamp = 2023-05-19T12:50:03.153629-05:00
│   │   ├── 4
│   │   │   ├── hyperspy_version = 1.7.5
│   │   │   ├── io_plugin = hyperspy.io_plugins.zspy
│   │   │   ├── operation = save
│   │   │   └── timestamp = 2023-05-19T12:55:50.994159-05:00
│   │   └── 5
│   │       ├── hyperspy_version = 1.7.5
│   │       ├── io_plugin = hyperspy.io_plugins.zspy
│   │       ├── operation = load
│   │       └── timestamp = 2023-05-19T22:48:50.997178-05:00
│   └── title = MgO Nano-Crystals
└── Signal
    └── signal_type = electron_diffraction

Inspect the data type of the object

dtype('float32')

Inspect the metadata associated with the object 'dp'

├── General
│   ├── FileIO
│   │   ├── 0
│   │   │   ├── hyperspy_version = 1.7.5
│   │   │   ├── io_plugin = hyperspy.io_plugins.hspy
│   │   │   ├── operation = load
│   │   │   └── timestamp = 2023-05-19T12:48:34.428982-05:00
│   │   ├── 1
│   │   │   ├── hyperspy_version = 1.7.5
│   │   │   ├── io_plugin = hyperspy.io_plugins.zspy
│   │   │   ├── operation = save
│   │   │   └── timestamp = 2023-05-19T12:48:37.501801-05:00
│   │   ├── 2
│   │   │   ├── hyperspy_version = 1.7.5
│   │   │   ├── io_plugin = hyperspy.io_plugins.zspy
│   │   │   ├── operation = save
│   │   │   └── timestamp = 2023-05-19T12:49:11.044073-05:00
│   │   ├── 3
│   │   │   ├── hyperspy_version = 1.7.5
│   │   │   ├── io_plugin = hyperspy.io_plugins.zspy
│   │   │   ├── operation = save
│   │   │   └── timestamp = 2023-05-19T12:50:03.153629-05:00
│   │   ├── 4
│   │   │   ├── hyperspy_version = 1.7.5
│   │   │   ├── io_plugin = hyperspy.io_plugins.zspy
│   │   │   ├── operation = save
│   │   │   └── timestamp = 2023-05-19T12:55:50.994159-05:00
│   │   └── 5
│   │       ├── hyperspy_version = 1.7.5
│   │       ├── io_plugin = hyperspy.io_plugins.zspy
│   │       ├── operation = load
│   │       └── timestamp = 2023-05-19T22:48:50.997178-05:00
│   └── title = MgO Nano-Crystals
└── Signal
    └── signal_type = electron_diffraction

Set important experimental parameters using the built in function

In [14]:
dp.set_experimental_parameters(beam_energy=300.0,
                               camera_length=21.0,
                               scan_rotation=277.0,
                               convergence_angle=0.7,
                               exposure_time=10.0)

See how this changed the metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── Diffraction
│       │       ├── camera_length = 21.0
│       │       └── exposure_time = 10.0
│       ├── beam_energy = 300.0
│       ├── convergence_angle = 0.7
│       └── scan_rotation = 277.0
├── General
│   ├── FileIO
│   │   ├── 0
│   │   │   ├── hyperspy_version = 1.7.5
│   │   │   ├── io_plugin = hyperspy.io_plugins.hspy
│   │   │   ├── operation = load
│   │   │   └── timestamp = 2023-05-19T12:48:34.428982-05:00
│   │   ├── 1
│   │   │   ├── hyperspy_version = 1.7.5
│   │   │   ├── io_plugin = hyperspy.io_plugins.zspy
│   │   │   ├── operation = save
│   │   │   └── timestamp = 2023-05-19T12:48:37.501801-05:00
│   │   ├── 2
│   │   │   ├── hyperspy_version = 1.7.5
│   │   │   ├── io_plugin = hyperspy.io_plugins.zspy
│   │   │   ├── operation = save
│   │   │   └── timestamp = 2023-05-19T12:49:11.044073-05:00
│   │   ├── 3
│   │   │   ├── hyperspy_version = 1.7.5
│   │   │   ├── io_plugin = hyperspy.io_plugins.zspy
│   │   │   ├── operation = save
│   │   │   └── timestamp = 2023-05-19T12:50:03.153629-05:00
│   │   ├── 4
│   │   │   ├── hyperspy_version = 1.7.5
│   │   │   ├── io_plugin = hyperspy.io_plugins.zspy
│   │   │   ├── operation = save
│   │   │   └── timestamp = 2023-05-19T12:55:50.994159-05:00
│   │   └── 5
│   │       ├── hyperspy_version = 1.7.5
│   │       ├── io_plugin = hyperspy.io_plugins.zspy
│   │       ├── operation = load
│   │       └── timestamp = 2023-05-19T22:48:50.997178-05:00
│   └── title = MgO Nano-Crystals
└── Signal
    └── signal_type = electron_diffraction

Plot the data to inspect it

## 2. Alignment & Calibration

Let's center the direct beam for the dataset

In [16]:
# get the direct beam position using `get_direct_beam_position`


In [4]:
# compute the shifts


In [3]:
# plot the orginal shifts


In [2]:
# make the shifts into a linear plane


In [1]:
# plot the shifts again!


Align the dataset based on the direct beam position

#### Setting the Calibration
Set the calibration. This is usually known for some detector from a standard sample
or you can get this from the dataset if the scale is known.

In [23]:
scale = 0.03246
scale_real = 3.03
dp.set_diffraction_calibration(scale)
dp.set_scan_calibration(scale_real)

In [ ]:
# plot calibrated data

<a id='vdf'></a>

##  3. Virtual Diffraction Imaging & Selecting Regions

### 3.1 Interactive VDF Imaging

Plot an interactive virtual image integrating intensity within a circular subset of pixels in the diffraction pattern

In [25]:
# create the rois


In [26]:
# coompute the mean dp


In [5]:
# lets just add a custom VDF to the image to get both VDF and VBF images


In [6]:
# inspect the bf roi I like to save these values above for reproduceability.


In [7]:
# add df roi


In [8]:
# inspect the Df roi I like to save these values above for reproduceability.


In [9]:
# get the integrated intensity for the bf and df

Get the virtual diffraction image associated with the last integration window used interactively

In [32]:
# compute the two images

In [ ]:
# plot both virtual images

In [ ]:
# inspect the metadata

In [ ]:
# save the image

### 3.2 Azimuthal Integration

Pyxem uses the [`pyfai`](https://github.com/silx-kit/pyFAI/tree/v2023.1) library to handle azimuthal integrations including the effects of the Ewald Sphere.  Because of this you should set the calibration and the beam energy before integration.

For speed a AzimuthalIntegrator object is precomputed which reduces redundant calculations.

In [36]:
# set the unit and beam_energy


In [37]:
# set the Azimuthal Integrator


In [38]:
# get the 1D Azimuthal Integration


In [39]:
# compute the dataset


In [40]:
# plot the inverse to get the VDF as a function of radius 


### 3.3 Select a region in the scan

Plot the data with an adjustable marker indicating where to crop the scan region

In [41]:
reg = hs.roi.RectangularROI(left=50.,
                            top=100.,
                            right=100.,
                            bottom=300.)
dp.plot(cmap='inferno')
reg.add_widget(dp)

In [42]:
# crop the dataset

In [43]:
# calculate the mean from the selected area

In [10]:
# plot the mean

## 4. Unsupervised learning

Perform singular value decomposition (SVD) of the data

Obtain a "Scree plot" by plotting the fraction of variance described by each principal component

In [45]:
# SVD won't converage with zeros


In [46]:
# Lazy decompositions only work with the processes and threaded schedulers (not the distributed scheduler)
dpc.compute() # compute this to load it into memory

In [47]:
# Perform a SVD Decomposition


Decomposition info:
  normalize_poissonian_noise=True
  algorithm=SVD
  output_dimension=None
  centre=None


In [11]:
# plot_explained_variance_ratio


In [12]:
# perform NNMF

In [13]:
# plot decomposition results


<a id='vec'></a>

##  5. Peak Finding

Perform peak finding on all diffraction patterns in data

In [51]:
# find peaks in the 2D dataset
# This will immediately compute and return a BaseSignal
peaks = dp.find_peaks(method='difference_of_gaussian',
                       min_sigma=1.,
                       max_sigma=6.,
                       sigma_ratio=1.6,
                       threshold=0.04,
                       overlap=0.99,
                       interactive=False)

Check the peaks object type

In [52]:
from pyxem.signals import DiffractionVectors

In [54]:
# Convert the peaks found to a Diffraction Vectors Object
dv = DiffractionVectors.from_peaks(peaks, center=(72, 72), calibration=dp.axes_manager.signal_axes[0].scale)

Look at what's in the peaks object

In [14]:
# Plot the number of peaks found at each point


In [56]:
#Cluster the vectors
distance_threshold = 0.1
min_samples = 7
unique_peaks = dv.get_unique_vectors(method='DBSCAN',
                                     distance_threshold=distance_threshold,
                                     min_samples=min_samples)
print(np.shape(unique_peaks.data)[0], ' unique vectors were found.')

68  unique vectors were found.


In [57]:
# remove the zero beam
unique_peaks = unique_peaks.filter_magnitude(min_magnitude=.4,
                                   max_magnitude=np.inf)
print(np.shape(unique_peaks)[0], ' unique vectors.')


67  unique vectors.


In [58]:
# plot the transpose and the unique vectors


In [15]:
# create Virtual Images from the unique vectors


In [60]:
#compute the Virtual darkfield images from the peaks

In [62]:
min_distance = 10.5
min_size = 40
max_size = 1000
max_number_of_grains = 3000
marker_radius = 2
exclude_border = 2
threshold= 0.65

In [63]:
# test for the right watershed parameters
from pyxem.utils.segment_utils import separate_watershed
i = 21
sep_i = separate_watershed(
    VDFs.inav[i].data, min_distance=min_distance, min_size=min_size,
    max_size=max_size, max_number_of_grains=max_number_of_grains,
    exclude_border=exclude_border, marker_radius=marker_radius,
    threshold=threshold, plot_on=True)

In [64]:
# get VDF segments
segs = VDFs.get_vdf_segments(min_distance=min_distance,
                             min_size=min_size,
                             max_size = max_size,
                             max_number_of_grains = max_number_of_grains,
                             exclude_border=exclude_border,
                             marker_radius=marker_radius,
                             threshold=threshold)


/home/cssfrancis/hyperspy-bundle/lib/python3.10/site-packages/pyxem/signals/virtual_dark_field_image.py:110: UserWarning: Changed in version 0.15.0.  May cause unexpectederrors related to managing the proper axes.
  warnings.warn(


In [65]:
# plot the segments


Calculate normalised cross-correlations between all VDF image segments to identify those that are related to the same crystal.

In [16]:
# get cross correlation matrix

If the correlation value exceeds corr_threshold for certain segments, those segments are summed. These segments are discarded if the number of these segments are below vector_threshold, as this number corresponds to the number of detected diffraction peaks associated with the single crystal. The vector_threshold criteria is included to avoid including segment images resulting from noise or incorrect segmentation.

In [67]:
corr_threshold=0.3
vector_threshold=4
segment_threshold=2

In [68]:
corrsegs = segs.correlate_vdf_segments(
    corr_threshold=corr_threshold, vector_threshold=vector_threshold,
    segment_threshold=segment_threshold)
print(np.shape(corrsegs.segments)[0],' correlated segments were found.')

  0%|                                                    | 0/87 [00:00<?, ?it/s]


7  correlated segments were found.


In [17]:
# plot correlation segments